# Internal Dependencies
<br>  

### References
- [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html)
- [Calculate metrics](https://101.jqassistant.org/calculate-metrics/index.html)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plot
from neo4j import GraphDatabase

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))
driver.verify_connectivity()

In [3]:
def get_cypher_query_from_file(cypherFileName):
    with open(cypherFileName) as file:
        return ' '.join(file.readlines())

In [4]:
def query_cypher_to_data_frame(filename : str, limit: int = 10_000):
    cypher_query_template = "{query}\nLIMIT {row_limit}"
    cypher_query = get_cypher_query_from_file(filename)
    cypher_query = cypher_query_template.format(query = cypher_query, row_limit = limit)
    records, summary, keys = driver.execute_query(cypher_query)
    return pd.DataFrame([r.values() for r in records], columns=keys)

In [5]:
def query_first_non_empty_cypher_to_data_frame(*filenames : str, limit: int = 10_000):
    """
    Executes the Cypher queries of the given files and returns the first result that is not empty.
    If all given file names result in empty results, the last (empty) result will be returned.
    By additionally specifying "limit=" the "LIMIT" keyword will appended to query so that only the first results get returned.
    """    
    result=pd.DataFrame()
    for filename in filenames:
        result=query_cypher_to_data_frame(filename, limit)
        if not result.empty:
            return result
    return result

In [6]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [7]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

In [8]:
# Pandas DataFrame Display Configuration
pd.set_option('display.max_colwidth', 300)

## Artifacts

List the artifacts this notebook is based on. Different sorting variations help finding artifacts by their features and support larger code bases where the list of all artifacts gets too long.

Only the top 30 entries are shown. The whole table can be found in the following CSV report:  
`List_all_Java_artifacts`

In [9]:
artifacts = query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_all_Java_artifacts.cypher")

### Table 1a - Top 30 artifacts with the highest package count

In [10]:
# Sort by number of packages descending
artifacts.sort_values(by=['packages','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.11.0.jar,66,809,8,0
1,axon-server-connector-4.11.0.jar,11,142,1,4
2,axon-modelling-4.11.0.jar,10,158,6,1
3,axon-eventsourcing-4.11.0.jar,9,133,5,2
4,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
5,axon-test-4.11.0.jar,8,87,1,3
6,axon-configuration-4.11.0.jar,1,41,2,4
7,axon-disruptor-4.11.0.jar,1,22,1,3
8,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1


### Table 1b - Top 30 artifacts with the highest type count

In [11]:
# Sort by number of types descending
artifacts.sort_values(by=['types','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.11.0.jar,66,809,8,0
1,axon-modelling-4.11.0.jar,10,158,6,1
2,axon-server-connector-4.11.0.jar,11,142,1,4
3,axon-eventsourcing-4.11.0.jar,9,133,5,2
4,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
5,axon-test-4.11.0.jar,8,87,1,3
6,axon-configuration-4.11.0.jar,1,41,2,4
7,axon-disruptor-4.11.0.jar,1,22,1,3
8,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1


### Table 1c - Top 30 artifacts with the highest number of incoming dependencies

The following table lists the top 30 artifacts that are used the most by other artifacts (highest count of incoming dependencies, highest in-degree).

In [12]:
# Sort by number of incoming dependencies descending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.11.0.jar,66,809,8,0
1,axon-modelling-4.11.0.jar,10,158,6,1
2,axon-eventsourcing-4.11.0.jar,9,133,5,2
3,axon-configuration-4.11.0.jar,1,41,2,4
4,axon-disruptor-4.11.0.jar,1,22,1,3
5,axon-server-connector-4.11.0.jar,11,142,1,4
6,axon-test-4.11.0.jar,8,87,1,3
7,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
8,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7


### Table 1d - Top 30 artifacts with the highest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the highest number of other artifacts (highest count of outgoing dependencies, highest out-degree).

In [13]:
# Sort by number of outgoing dependencies descending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
1,axon-configuration-4.11.0.jar,1,41,2,4
2,axon-server-connector-4.11.0.jar,11,142,1,4
3,axon-disruptor-4.11.0.jar,1,22,1,3
4,axon-test-4.11.0.jar,8,87,1,3
5,axon-eventsourcing-4.11.0.jar,9,133,5,2
6,axon-modelling-4.11.0.jar,10,158,6,1
7,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
8,axon-messaging-4.11.0.jar,66,809,8,0


### Table 1e - Top 30 artifacts with the lowest package count

In [14]:
# Sort by number of packages ascending
artifacts.sort_values(by=['packages','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-configuration-4.11.0.jar,1,41,2,4
1,axon-disruptor-4.11.0.jar,1,22,1,3
2,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
3,axon-test-4.11.0.jar,8,87,1,3
4,axon-eventsourcing-4.11.0.jar,9,133,5,2
5,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
6,axon-modelling-4.11.0.jar,10,158,6,1
7,axon-server-connector-4.11.0.jar,11,142,1,4
8,axon-messaging-4.11.0.jar,66,809,8,0


### Table 1f - Top 30 artifacts with the lowest type count

In [15]:
# Sort by number of types ascending
artifacts.sort_values(by=['types','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
1,axon-disruptor-4.11.0.jar,1,22,1,3
2,axon-configuration-4.11.0.jar,1,41,2,4
3,axon-test-4.11.0.jar,8,87,1,3
4,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
5,axon-eventsourcing-4.11.0.jar,9,133,5,2
6,axon-server-connector-4.11.0.jar,11,142,1,4
7,axon-modelling-4.11.0.jar,10,158,6,1
8,axon-messaging-4.11.0.jar,66,809,8,0


### Table 1g - Top 30 artifacts with the lowest number of incoming dependencies

The following table lists the top 30 artifacts that are used the least by other artifacts (lowest count of incoming dependencies, lowest in-degree).

In [16]:
# Sort by number of incoming dependencies ascending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7
1,axon-disruptor-4.11.0.jar,1,22,1,3
2,axon-server-connector-4.11.0.jar,11,142,1,4
3,axon-test-4.11.0.jar,8,87,1,3
4,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
5,axon-configuration-4.11.0.jar,1,41,2,4
6,axon-eventsourcing-4.11.0.jar,9,133,5,2
7,axon-modelling-4.11.0.jar,10,158,6,1
8,axon-messaging-4.11.0.jar,66,809,8,0


### Table 1h - Top 30 artifacts with the lowest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the lowest number of other artifacts (lowest count of outgoing dependencies, lowest out-degree).

In [17]:
# Sort by number of outgoing dependencies ascending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.11.0.jar,66,809,8,0
1,axon-modelling-4.11.0.jar,10,158,6,1
2,axon-tracing-opentelemetry-4.11.0.jar,1,5,1,1
3,axon-eventsourcing-4.11.0.jar,9,133,5,2
4,axon-disruptor-4.11.0.jar,1,22,1,3
5,axon-test-4.11.0.jar,8,87,1,3
6,axon-configuration-4.11.0.jar,1,41,2,4
7,axon-server-connector-4.11.0.jar,11,142,1,4
8,axon-spring-boot-autoconfigure-4.11.0.jar,9,88,0,7


## Cyclic Dependencies

Cyclic dependencies occur when one package uses a class of another package and vice versa. 
These dependencies can lead to problems when one of these packages needs to be changed.

## Table 2a - Cyclic Dependencies Overview

Show the top 40 cyclic dependencies sorted by the most promising to resolve first. This is done by calculating the number of forward dependencies (first cycle participant to second cycle participant) in relation to backward dependencies (second cycle participant back to first cycle participant). The higher this rate (approaching 1), the easier it should be to resolve the cycle by focussing on the few backward dependencies.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies`

**Columns:**
- *artifactName* identifies the artifact of the first participant of the cycle
- *packageName* identifies the package of the first participant of the cycle
- *dependentArtifactName* identifies the artifact of the second participant of the cycle
- *dependentPackageName* identifies the package of the second participant of the cycle
- *forwardToBackwardBalance* is between 0 and 1. High for many forward and few backward dependencies.
- *numberForward* contains the number of dependencies from the first participant of the cycle to the second one
- *numberBackward* contains the number of dependencies from the second participant of the cycle back to the first one
- *someForwardDependencies* lists some forward dependencies in the text format "type1 -> type2"
- *backwardDependencies* lists the backward dependencies in the format "type1 <- type2" that are recommended to get resolved

In [18]:
cyclic_dependencies = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies.cypher")
cyclic_dependencies.head(40)

,artifactName,packageName,dependentArtifactName,dependentPackageName,forwardToBackwardBalance,numberForward,numberBackward,someForwardDependencies,backwardDependencies
0,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,0.900000,19,1,"[DefaultEventProcessorSpanFactory->NoOpSpanFactory$NoOpSpan, DefaultEventProcessorSpanFactory->Span, DefaultEventProcessorSpanFactory->SpanFactory, DefaultEventProcessorSpanFactory$Builder->SpanFactory, DefaultEventBusSpanFactory->SpanFactory, DefaultEventBusSpanFactory->Span, SimpleEventBus$Bui...",[NestingSpanFactory->EventMessage]
1,axon-messaging-4.11.0,org.axonframework.queryhandling,axon-messaging-4.11.0,org.axonframework.messaging.responsetypes,0.882353,16,1,"[QueryGateway->ResponseType, QueryGateway->ResponseTypes, SubscriptionQueryMessage->ResponseType, QueryMessage->ResponseType, GenericQueryMessage->ResponseType, GenericSubscriptionQueryMessage->ResponseType, SimpleQueryUpdateEmitter->ResponseType, SimpleQueryUpdateEmitter->OptionalResponseType, ...",[ConvertingResponseMessage->QueryResponseMessage]
2,axon-messaging-4.11.0,org.axonframework.queryhandling,axon-messaging-4.11.0,org.axonframework.tracing,0.875000,15,1,"[DefaultQueryBusSpanFactory$Builder->SpanFactory, DefaultQueryUpdateEmitterSpanFactory->SpanFactory, DefaultQueryUpdateEmitterSpanFactory->Span, SimpleQueryBus$Builder->SpanFactory, SimpleQueryBus$Builder->NoOpSpanFactory, SimpleQueryUpdateEmitter->Span, DefaultQueryUpdateEmitterSpanFactory$Buil...",[SpanUtils->QueryMessage]
3,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging,0.857143,39,3,"[ConcludesBatchParameterResolverFactory->Message, DefaultEventProcessorSpanFactory->Message, EventMessageHandler->MessageHandler, EventMessageHandler->Message, TrackingEventProcessorConfiguration->StreamableMessageSource, ReplayToken->Message, AnnotationEventHandlerAdapter->Message, SequenceNumb...","[Headers->DomainEventMessage, StreamableMessageSource->TrackingToken, Headers->EventMessage]"
4,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging.annotation,0.840000,23,2,"[ConcludesBatchParameterResolverFactory->ParameterResolver, ConcludesBatchParameterResolverFactory->AbstractAnnotatedParameterResolverFactory, ResetHandler->MessageHandler, AnnotationEventHandlerAdapter->AnnotatedHandlerInspector, AnnotationEventHandlerAdapter->ClasspathParameterResolverFactory,...","[SourceIdParameterResolverFactory$SourceIdParameterResolver->DomainEventMessage, AggregateTypeParameterResolverFactory$AggregateTypeParameterResolver->DomainEventMessage]"
5,axon-messaging-4.11.0,org.axonframework.deadline,axon-messaging-4.11.0,org.axonframework.tracing,0.800000,9,1,"[DefaultDeadlineManagerSpanFactory->Span, DefaultDeadlineManagerSpanFactory->SpanFactory, SimpleDeadlineManager$Builder->SpanFactory, SimpleDeadlineManager$Builder->NoOpSpanFactory, DefaultDeadlineManagerSpanFactory$Builder->SpanFactory, SimpleDeadlineManager->Span, DeadlineManagerSpanFactory->S...",[SpanUtils->DeadlineMessage]
6,axon-messaging-4.11.0,org.axonframework.commandhandling,axon-messaging-4.11.0,org.axonframework.tracing,0.777778,8,1,"[DefaultCommandBusSpanFactory->SpanFactory, DefaultCommandBusSpanFactory->Span, DefaultCommandBusSpanFactory$Builder->SpanFactory, CommandBusSpanFactory->Span, AsynchronousCommandBus$Builder->SpanFactory, SimpleCommandBus$Builder->SpanFactory, SimpleCommandBus$Builder->NoOpSpanFactory, SimpleCom...",[SpanUtils->CommandMessage]
7,axon-eventsourcing-4.11.0,org.axonframework.eventsourcing,axon-eventsourcing-4.11.0,org.axonframework.eventsourcing.eventstore,0.777778,16,2,"[AbstractSnapshotter->DomainEventStream, AbstractSnapshotter->EventStore, FilteringEventStorageEngine->EventStorageEngine, FilteringEventStorageEngine->DomainEventStream, EventSourcingRepository$Builder->EventStore, AbstractSnapshotter$CreateSnapshotTask->DomainEventStream, AbstractSna

### Table 2b - Cyclic Dependencies Break Down

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown`

**Columns in addition to Table 2a:**
- *dependency* shows the cycle dependency in the text format "type1 -> type2" (forward) or "type2<-type1" (backward)

In [19]:
cyclic_dependencies_breakdown = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown.cypher",limit=40)
cyclic_dependencies_breakdown

,artifactName,packageName,dependentArtifactName,dependentPackageName,dependency,forwardToBackwardBalance,numberForward,numberBackward
0,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,DefaultEventProcessorSpanFactory$Builder->SpanFactory,0.900000,19,1
1,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,SubscribingEventProcessor$Builder->SpanFactory,0.900000,19,1
2,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,DefaultEventBusSpanFactory$Builder->SpanFactory,0.900000,19,1
3,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,SimpleEventBus$Builder->SpanFactory,0.900000,19,1
4,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,EventMessage<-NestingSpanFactory,0.900000,19,1
5,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,EventProcessorSpanFactory->Span,0.900000,19,1
6,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,DefaultEventBusSpanFactory->Span,0.900000,19,1
7,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,AbstractEventProcessor->Span,0.900000,19,1
8,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,AbstractEventProcessor$Builder->SpanFactory,0.900000,19,1
9,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,AbstractEventBus$Builder->SpanFactory,0.900000,19,1


### Table 2c - Cyclic Dependencies Break Down - Backward Dependencies Only

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first. This table only contains the backward dependencies from the second participant of the cycle back to the first one that are the most promising to resolve.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown_BackwardOnly`

In [20]:
cyclic_dependencies_breakdown_backward = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown_Backward_Only.cypher",limit=40)
cyclic_dependencies_breakdown_backward

,artifactName,packageName,dependentArtifactName,dependentPackageName,dependency,forwardToBackwardBalance,numberForward,numberBackward
0,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.tracing,EventMessage<-NestingSpanFactory,0.900000,19,1
1,axon-messaging-4.11.0,org.axonframework.queryhandling,axon-messaging-4.11.0,org.axonframework.messaging.responsetypes,QueryResponseMessage<-ConvertingResponseMessage,0.882353,16,1
2,axon-messaging-4.11.0,org.axonframework.queryhandling,axon-messaging-4.11.0,org.axonframework.tracing,QueryMessage<-SpanUtils,0.875000,15,1
3,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging,DomainEventMessage<-Headers,0.857143,39,3
4,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging,EventMessage<-Headers,0.857143,39,3
5,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging,TrackingToken<-StreamableMessageSource,0.857143,39,3
6,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging.annotation,DomainEventMessage<-AggregateTypeParameterResolverFactory$AggregateTypeParameterResolver,0.840000,23,2
7,axon-messaging-4.11.0,org.axonframework.eventhandling,axon-messaging-4.11.0,org.axonframework.messaging.annotation,DomainEventMessage<-SourceIdParameterResolverFactory$SourceIdParameterResolver,0.840000,23,2
8,axon-messaging-4.11.0,org.axonframework.deadline,axon-messaging-4.11.0,org.axonframework.tracing,DeadlineMessage<-SpanUtils,0.800000,9,1
9,axon-messaging-4.11.0,org.axonframework.commandhandling,axon-messaging-4.11.0,org.axonframework.tracing,CommandMessage<-SpanUtils,0.777778,8,1


## Interface Segregation Candidates

Well known from [Design Principles and Design Patterns by Robert C. Martin](http://staff.cs.utu.fi/~jounsmed/doos_06/material/DesignPrinciplesAndPatterns.pdf), the *Interface Segregation Principle* suggests that software components should have narrow, focused interfaces rather than large, general-purpose ones. The goal is to minimize the dependencies between components and increase modularity, flexibility, and maintainability.

Smaller, focused and purpose-driven interfaces

- make it easier to modify individual components without affecting the rest of the system.
- make it clearer which client is affected by which change.
- don’t force their clients to depend on methods they don’t need.
- reduce the scope of changes since a change to one component doesn’t affect others.
- lead to a more loosely coupled architecture that is easier to understand and maintain.

Reference: [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html#interface-segregation)

### How to apply the results

If just one method of a type is used, especially in many places, then the result of this method can be used to call e.g. a method or constuct an object instead of using the whole object and then just calling that single method.

If there are a couple of methods that are used for a distinct purpose, those could be factored out into a separate interface. The original type can extended/implement the new interface so that there are no breaking changes. Then all the callers, that use only this group of methods, can be changed to the new interface.


### Table 4 - Top 40 most used combinations of methods

The following table shows the top 40 most used combinations of methods of larger types that might benefit from applying the *Interface Segregation Principle*. The whole table can be found in the CSV report `Candidates_for_Interface_Segregation`.

In [21]:
interface_segregation_candidates=query_cypher_to_data_frame("../cypher/Internal_Dependencies/Candidates_for_Interface_Segregation.cypher", limit=40)
interface_segregation_candidates

,fullDependentTypeName,declaredMethods,calledMethodNames,calledMethods,callerTypes
0,org.axonframework.commandhandling.CommandMessage,9,[getCommandName],1,20
1,org.axonframework.eventhandling.EventMessage,9,"[getTimestamp, getIdentifier]",2,10
2,org.axonframework.eventhandling.EventMessage,9,[getIdentifier],1,10
3,org.axonframework.eventhandling.DomainEventMessage,10,[getSequenceNumber],1,9
4,org.axonframework.eventhandling.TrackedEventMessage,10,[trackingToken],1,8
5,org.axonframework.commandhandling.GenericCommandResultMessage,14,[asCommandResultMessage],1,6
6,org.axonframework.eventhandling.DomainEventMessage,10,"[getSequenceNumber, getAggregateIdentifier, getType]",3,6
7,org.axonframework.messaging.ResultMessage,9,"[isExceptional, exceptionResult]",2,6
8,org.axonframework.eventhandling.ReplayToken,13,[createReplayToken],1,5
9,org.axonframework.deadline.GenericDeadlineMessage,11,[asDeadlineMessage],1,5


## Package Usage

### Table 5 - Types that are used by multiple packages

This table shows the top 40 packages that are used by the highest number of different packages. The whole table can be found in the CSV report `List_types_that_are_used_by_many_different_packages`.


In [22]:
types_used_by_many_packages=query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_types_that_are_used_by_many_different_packages.cypher", limit=40)
types_used_by_many_packages

,fullQualifiedDependentTypeName,dependentTypeName,dependentTypeLabels,numberOfUsingPackages
0,org.axonframework.common.BuilderUtils,BuilderUtils,"[Type, File, Java, Class, ByteCode, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWeaklyConnectedComponent0, Ma...",50
1,org.axonframework.common.AxonConfigurationException,AxonConfigurationException,"[Type, File, Java, Class, ByteCode, Throwable, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWeaklyConnectedCom...",43
2,org.axonframework.messaging.Message,Message,"[Type, File, Java, ByteCode, GenericDeclaration, Interface, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityBetweenness, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInduce...",42
3,org.axonframework.messaging.MetaData,MetaData,"[Type, File, Java, Class, ByteCode, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityBetweenness, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4Ty...",40
4,org.axonframework.serialization.Serializer,Serializer,"[Type, File, Java, ByteCode, Interface, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityBetweenness, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mar...",37
5,org.axonframework.eventhandling.EventMessage,EventMessage,"[Type, File, Java, ByteCode, GenericDeclaration, Interface, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWeakl...",36
6,org.axonframework.messaging.unitofwork.UnitOfWork,UnitOfWork,"[Type, File, Java, ByteCode, GenericDeclaration, Interface, Mark4TopCentralityArticleRank, Mark4TopCentralityBetweenness, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWe...",33
7,org.axonframework.common.transaction.TransactionManager,TransactionManager,"[Type, File, Java, ByteCode, Interface, Mark4TopCentralityArticleRank, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWeaklyConnectedComponent0, Mark4TypeLabelPropagation9...",31
8,org.axonframework.common.Assert,Assert,"[Type, File, Java, Class, ByteCode, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInducedTopicSearchHub, Mark4TypeWeaklyConnectedComponent0, Ma...",29
9,org.axonframework.serialization.SerializedObject,SerializedObject,"[Type, File, Java, ByteCode, GenericDeclaration, Interface, Mark4TopCentralityPageRank, Mark4TopCentralityArticleRank, Mark4TopCentralityBetweenness, Mark4TopCentralityHarmonic, Mark4TopCentralityCloseness, Mark4TopCentralityHyperlinkInducedTopicSearchAuthority, Mark4TopCentralityHyperlinkInduce...",28


### Table 6 - Packages that are used by multiple artifacts

This table shows the top 30 artifacts that only use a few (compared to all existing) packages of another artifact.
The whole table can be found in the CSV report `ArtifactPackageUsage`.

In [23]:
used_packages_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_packages_compared_to_all_existing_are_used_by_dependent_artifacts.cypher",limit=30)
used_packages_of_dependent_artifact

,artifactName,dependentArtifactName,dependentPackages,dependentArtifactPackages,packageUsagePercentage,dependentFullQualifiedPackageNames,dependentPackageNames
0,axon-tracing-opentelemetry-4.11.0,axon-messaging-4.11.0,3,66,0.045455,"[org.axonframework.tracing, org.axonframework.common, org.axonframework.messaging]","[tracing, common, messaging]"
1,axon-server-connector-4.11.0,axon-modelling-4.11.0,1,10,0.100000,[org.axonframework.modelling.command],[command]
2,axon-spring-boot-autoconfigure-4.11.0,axon-test-4.11.0,1,8,0.125000,[org.axonframework.test.server],[server]
3,axon-disruptor-4.11.0,axon-messaging-4.11.0,9,66,0.136364,"[org.axonframework.common.caching, org.axonframework.common, org.axonframework.monitoring, org.axonframework.commandhandling.callbacks, org.axonframework.commandhandling, org.axonframework.common.transaction, org.axonframework.messaging, org.axonframework.messaging.unitofwork, org.axonframework....","[caching, common, monitoring, callbacks, commandhandling, transaction, messaging, unitofwork, annotation]"
4,axon-test-4.11.0,axon-messaging-4.11.0,10,66,0.151515,"[org.axonframework.common, org.axonframework.eventhandling, org.axonframework.eventhandling.scheduling, org.axonframework.messaging.annotation, org.axonframework.commandhandling.gateway, org.axonframework.messaging, org.axonframework.deadline, org.axonframework.commandhandling, org.axonframework...","[common, eventhandling, scheduling, annotation, gateway, messaging, deadline, commandhandling, unitofwork, stream]"
5,axon-disruptor-4.11.0,axon-modelling-4.11.0,2,10,0.200000,"[org.axonframework.modelling.command, org.axonframework.modelling.command.inspection]","[command, inspection]"
6,axon-eventsourcing-4.11.0,axon-modelling-4.11.0,2,10,0.200000,"[org.axonframework.modelling.command, org.axonframework.modelling.command.inspection]","[command, inspection]"
7,axon-test-4.11.0,axon-eventsourcing-4.11.0,2,9,0.222222,"[org.axonframework.eventsourcing.eventstore, org.axonframework.eventsourcing]","[eventstore, eventsourcing]"
8,axon-disruptor-4.11.0,axon-eventsourcing-4.11.0,2,9,0.222222,"[org.axonframework.eventsourcing.eventstore, org.axonframework.eventsourcing]","[eventstore, eventsourcing]"
9,axon-modelling-4.11.0,axon-messaging-4.11.0,18,66,0.272727,"[org.axonframework.eventhandling, org.axonframework.commandhandling, org.axonframework.messaging.unitofwork, org.axonframework.messaging, org.axonframework.common.property, org.axonframework.messaging.annotation, org.axonframework.common, org.axonframework.common.annotation, org.axonframework.tr...","[eventhandling, commandhandling, unitofwork, messaging, property, annotation, common, tracing, serialization, xml, jpa, caching, lock, legacyjpa, interceptors, deadline, jdbc]"


### Table 7 - Types that are used by multiple artifacts

This table shows the top 30 types that only use a few (compared to all existing) types of another artifact. The whole table can be found in the CSV report `ClassesPerPackageUsageAcrossArtifacts`.

In [24]:
used_types_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_classes_compared_to_all_existing_in_the_same_package_are_used_by_dependent_packages_across_different_artifacts.cypher", limit=30)
used_types_of_dependent_artifact

,artifactName,dependentArtifactName,packageName,dependentPackage.fqn,dependentTypes,dependentPackageTypes,typeUsagePercentage,dependentTypeNames
0,axon-test-4.11.0,axon-messaging-4.11.0,org.axonframework.test.matchers,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
1,axon-modelling-4.11.0,axon-messaging-4.11.0,org.axonframework.modelling.saga.metamodel,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
2,axon-spring-boot-autoconfigure-4.11.0,axon-messaging-4.11.0,org.axonframework.springboot.autoconfig.legacyjpa,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventBus]
3,axon-server-connector-4.11.0,axon-messaging-4.11.0,org.axonframework.axonserver.connector,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventBusSpanFactory]
4,axon-eventsourcing-4.11.0,axon-messaging-4.11.0,org.axonframework.eventsourcing.snapshotting,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.DomainEventData]
5,axon-spring-boot-autoconfigure-4.11.0,axon-messaging-4.11.0,org.axonframework.springboot.util,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
6,axon-server-connector-4.11.0,axon-modelling-4.11.0,org.axonframework.axonserver.connector,org.axonframework.modelling.command,1,56,0.017857,[org.axonframework.modelling.command.ConcurrencyException]
7,axon-eventsourcing-4.11.0,axon-modelling-4.11.0,org.axonframework.eventsourcing.eventstore.jdbc,org.axonframework.modelling.command,1,56,0.017857,[org.axonframework.modelling.command.ConcurrencyException]
8,axon-eventsourcing-4.11.0,axon-modelling-4.11.0,org.axonframework.eventsourcing.conflictresolution,org.axonframework.modelling.command,1,56,0.017857,[org.axonframework.modelling.command.ConflictingAggregateVersionException]
9,axon-test-4.11.0,axon-messaging-4.11.0,org.axonframework.test.eventscheduler,org.axonframework.eventhandling,2,100,0.020000,"[org.axonframework.eventhandling.EventMessage, org.axonframework.eventhandling.GenericEventMessage]"


### Table 8 - Duplicate package names across artifacts

This table shows the top 30 duplicate package names across artifacts. They are ordered by the number of duplicates descending.

This might lead to confusion, makes importing more error prone and might even lead to duplicate classes where only one of them will be loaded by the class loader. If a package is named the same way in two or more artifacts this even allows another artifact to access package protected classes, methods or members which might not be intended. 

The whole table can be found in the CSV report `DuplicatePackageNamesAcrossArtifacts`.

In [25]:
duplicate_package_names_across_artifacts=query_cypher_to_data_frame("../cypher/Artifact_Dependencies/Artifacts_with_duplicate_packages.cypher", limit=30)
duplicate_package_names_across_artifacts

,packageName,duplicates,artifactNames


### Table 9 - Annotated elements

This table shows 30 most used Java Annotations including some examples where they are used.


In [26]:
annotated_elements=query_cypher_to_data_frame("../cypher/Java/Annotated_code_elements.cypher", limit=30)
annotated_elements

,annotationName,languageElement,numberOfAnnotatedElements,examples
0,javax.annotation.Nonnull,Parameter,1668,"[org.axonframework.disruptor.commandhandling.BlacklistDetectingCallback.onResult(0), org.axonframework.disruptor.commandhandling.BlacklistDetectingCallback.onResult(1), org.axonframework.disruptor.commandhandling.CommandHandlerInvoker$DisruptorRepository.load(0), org.axonframework.disruptor.comm..."
1,java.lang.Deprecated,Method,133,"[org.axonframework.modelling.command.AbstractRepository$Builder.spanFactory, org.axonframework.modelling.command.AggregateIdentifier.routingKey, org.axonframework.modelling.command.GenericJpaRepository$Builder.spanFactory, org.axonframework.modelling.command.LockingRepository$Builder.spanFactory..."
2,org.springframework.context.annotation.Bean,Method,122,"[org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxEventStoreAutoConfiguration.eventStorageEngine, org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxAutoConfiguration.entityManagerProvider, org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxAutoConfiguration.tokenStore, or..."
3,org.springframework.boot.autoconfigure.condition.ConditionalOnMissingBean,Method,78,"[org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxAutoConfiguration.entityManagerProvider, org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxAutoConfiguration.tokenStore, org.axonframework.springboot.autoconfig.legacyjpa.JpaJavaxAutoConfiguration.sagaStore, org.axonframework.spr..."
4,java.lang.FunctionalInterface,Interface,65,"[org.axonframework.modelling.command.CreationPolicyAggregateFactory, org.axonframework.modelling.command.RepositoryProvider, org.axonframework.modelling.command.LockAwareAggregate$LockSupplier, org.axonframework.eventsourcing.conflictresolution.ConflictExceptionSupplier, org.axonframework.events..."
5,javax.annotation.Nullable,Parameter,62,"[org.axonframework.modelling.command.CreationPolicyAggregateFactory.create(0), org.axonframework.modelling.command.NoArgumentConstructorCreationPolicyAggregateFactory.create(0), org.axonframework.modelling.command.Repository.load(1), org.axonframework.modelling.command.inspection.ChildForwarding..."
6,javax.annotation.Nonnull,Method,60,"[org.axonframework.disruptor.commandhandling.DisruptorCommandBus.registerDispatchInterceptor, org.axonframework.modelling.command.CreationPolicyAggregateFactory.create, org.axonframework.modelling.command.NoArgumentConstructorCreationPolicyAggregateFactory.create, org.axonframework.modelling.com..."
7,java.lang.annotation.Target,Annotation,44,"[org.axonframework.modelling.command.AggregateIdentifier, org.axonframework.modelling.command.EntityId, org.axonframework.modelling.command.AggregateMember, org.axonframework.modelling.command.AggregateRoot, org.axonframework.modelling.command.AggregateVersion, org.axonframework.modelling.comman..."
8,java.lang.annotation.Retention,Annotation,44,"[org.axonframework.modelling.command.AggregateIdentifier, org.axonframework.modelling.command.EntityId, org.axonframework.modelling.command.AggregateMember, org.axonframework.modelling.command.AggregateRoot, org.axonframework.modelling.command.AggregateVersion, org.axonframework.modelling.comman..."
9,java.lang.Deprecated,Class,43,"[org.axonframework.modelling.command.legacyjpa.GenericJpaRepository, org.axonframework.modelling.saga.repository.jpa.AbstractSagaEntry, org.axonframework.modelling.saga.repository.legacyjpa.JpaSagaStore, org.axonframework.eventsourcing.MultiStreamableMessageSource$Builder, org.axonframework.even..."


### Table 10 - Distance distribution between dependent files

This table shows the file directory distance distribution between dependent files. Intuitively, the distance is given by the fewest number of change directory commands needed to navigate between a file and a dependency it uses. Those are aggregate to see how many dependent files are in the same directory, how many are just one change directory command apart, and so on.

In [27]:
query_first_non_empty_cypher_to_data_frame("../cypher/Internal_Dependencies/Get_file_distance_as_shortest_contains_path_for_dependencies.cypher",
                                           "../cypher/Internal_Dependencies/Set_file_distance_as_shortest_contains_path_for_dependencies.cypher", limit=20)

,dependency.fileDistanceAsFewestChangeDirectoryCommands,numberOfDependencies,numberOfDependencyUsers,numberOfDependencyProviders,examples
0,0,2685,1062,1195,"[/axon-configuration-4.11.0.jar uses /axon-disruptor-4.11.0.jar, /org/axonframework/disruptor/commandhandling/EventPublisher.class uses /org/axonframework/disruptor/commandhandling/AggregateBlacklistedException.class, /org/axonframework/disruptor/commandhandling/BlacklistDetectingCallback.class ..."
1,1,92,84,38,"[/org/axonframework/modelling/command/legacyjpa uses /org/axonframework/modelling/command, /org/axonframework/modelling/command/inspection uses /org/axonframework/modelling/command, /org/axonframework/modelling/command uses /org/axonframework/modelling/command/inspection, /org/axonframework/mode..."
2,2,2559,778,477,"[/org/axonframework/modelling/command/GenericJpaRepository$Builder.class uses /org/axonframework/modelling/command/inspection/AggregateModel.class, /org/axonframework/modelling/command/legacyjpa/GenericJpaRepository$Builder.class uses /org/axonframework/modelling/command/inspection/AggregateMode..."
3,4,2629,456,550,"[/org/axonframework/config uses /org/axonframework/disruptor/commandhandling, /org/axonframework/config/AggregateConfigurer.class uses /org/axonframework/disruptor/commandhandling/DisruptorCommandBus.class, /org/axonframework/springboot/autoconfig uses /org/axonframework/tracing/opentelemetry, /..."
